# 데이터 셋 만들기

1. iso 표준 데이터 다운로드

2. iso 표준 데이터에 맞춰 kaggle에서 받은 국가별 확진자 데이터의 국가명을 국가코드로 변경
    - 이 과정에서 iso 표준의 국가명과 괄호 또는 영문표기 등으로 맞지 않는 60여개의 국가는 수기로 변경
    - 코소보는 검색을 통해 XK로 변경
    - 데이터에 Diamond Princess, MS Zaandam, West Bank and Gaza 와 같은 국가가 아닌 발병지가 있어서 삭제
3. 공모전에서 제공하는 roamming 데이터를 iso 별로 일일 로밍 수 생성

4. 데이터 통합

In [205]:
import pandas as pd
import numpy as np

# 1. iso 표준 데이터

In [242]:
iso = pd.read_csv('./data/iso.csv')

In [243]:
# covide-19 글로벌 kaggle 데이터
kag = pd.read_csv('./data/corona-virus-report/covid_19_clean_complete.csv')
kag['Date'] = pd.to_datetime(kag['Date'])

In [244]:
kag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27456 entries, 0 to 27455
Data columns (total 8 columns):
Province/State    8320 non-null object
Country/Region    27456 non-null object
Lat               27456 non-null float64
Long              27456 non-null float64
Date              27456 non-null datetime64[ns]
Confirmed         27456 non-null int64
Deaths            27456 non-null int64
Recovered         27456 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 1.7+ MB


In [245]:
pivoted_kag = kag.pivot_table(index='Date', columns='Country/Region', values = 'Confirmed', aggfunc='sum')

In [246]:
col_list = list(pivoted_kag.columns)

In [247]:
iso.head()

,English short name,Alpha-2 code,Numeric
0,Afghanistan,AF,4.0
1,Albania,AL,8.0
2,Algeria,DZ,12.0
3,American Samoa,AS,16.0
4,Andorra,AD,20.0


In [248]:
tmp = []
iso_list = list(iso['English short name'])
iso_code_list = list(iso['Alpha-2 code'])
for i in col_list:
    if i in iso_list:
        index_num = iso_list.index(i)
        pivoted_kag['{}'.format(iso_code_list[index_num])] = pivoted_kag[i]
        pivoted_kag.drop([i], axis=1, inplace=True)
        
    else:
        tmp.append(i)

In [249]:
# 맞지 않는 국가 코드 딕셔너리 생성
etc_dict = {
 'Bahamas' : 'BS',
 'Bolivia' : 'BO',
 'Brunei' : 'BN',
 'Burma' : 'MM',
 'Central African Republic' : 'CF',
 'Comoros' : 'KM',
 'Congo (Brazzaville)' : 'CG',
 'Congo (Kinshasa)' : 'CD',
 "Cote d'Ivoire" : 'CI',
 'Dominican Republic' : 'DO',
 'Gambia' : 'GM',
 'Holy See' : 'VA',
 'Iran' : 'IR',
 'Kosovo' : 'XK',
 'Laos' : 'LA',
 'Moldova' : 'MD',
 'Netherlands' : 'NL',
 'Niger' : 'NE',
 'Philippines' : 'PH',
 'Russia' : 'RU',
 'South Korea' : 'KR',
 'Sudan' : 'SS',
 'Syria' : 'SY',
 'Taiwan*' : 'TW',
 'Tanzania' : 'TZ',
 'US' : 'US',
 'United Arab Emirates':'AE',
 'United Kingdom':'GB',
 'Venezuela':'VE',
 'Vietnam':'VN',
}

In [250]:
pivoted_kag.rename(columns=etc_dict, inplace=True)

In [251]:
pivoted_kag.drop(['Diamond Princess', 'MS Zaandam', 'West Bank and Gaza'], axis=1, inplace=True)

In [252]:
last_cols = list(pivoted_kag.columns)

In [253]:
# 일일 별 확진자 수로 컬럼 변경
for i in last_cols:
    pivoted_kag[i] = pivoted_kag[i].diff()

In [254]:
pivoted_kag.reset_index(inplace=True)

In [255]:
pivoted_kag.head()

Country/Region,Date,BS,BO,BN,MM,CF,KM,CG,CD,CI,...,TN,TR,UG,UA,UY,UZ,EH,YE,ZM,ZW
0,2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [256]:
pivoted_kag.iloc[0,1:] = 0

# 3. 로밍데이터

In [258]:
# 로밍데이터
roam1 = pd.read_csv('./data/corona_contest_data_0406/2. Roaming_data.csv')
roam2 = pd.read_csv('./data/corona_contest_data_0429/2. Roaming_data.csv')

In [259]:
roam = pd.concat([roam1, roam2])
roam.rename(columns={'return':'Date'}, inplace=True)
roam['Date'] = pd.to_datetime(roam['Date'], format='%Y%m%d')

In [260]:
# iso 편의를 위해 대문자로 변경

roam['iso'] = roam['iso'].str.upper() 

In [261]:
pivoted_roam = roam.pivot_table(index = 'Date', columns='iso', values='count', aggfunc='sum')

In [262]:
# index 리셋
pivoted_roam.reset_index(inplace=True)
# 결측치 0으로 대체
pivoted_roam.fillna(0, inplace=True)

In [263]:
pivoted_kag.head()

Country/Region,Date,BS,BO,BN,MM,CF,KM,CG,CD,CI,...,TN,TR,UG,UA,UY,UZ,EH,YE,ZM,ZW
0,2020-01-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [264]:
pivoted_roam.head()

iso,Date,AD,AE,AF,AL,AM,AN,AO,AR,AT,...,US,UY,UZ,VN,VU,WS,YE,ZA,ZM,ZW
0,2019-12-01,0.0,201.0,3.0,3.0,0.0,0.0,0.0,3.0,215.0,...,800.0,3.0,3.0,2328.0,0.0,0.0,0.0,9.0,9.0,12.0
1,2019-12-02,0.0,140.0,3.0,3.0,3.0,0.0,0.0,0.0,113.0,...,642.0,3.0,24.0,2470.0,3.0,0.0,0.0,3.0,3.0,0.0
2,2019-12-03,0.0,192.0,0.0,3.0,9.0,0.0,0.0,6.0,61.0,...,744.0,13.0,48.0,2540.0,0.0,0.0,0.0,18.0,20.0,21.0
3,2019-12-04,0.0,198.0,0.0,6.0,6.0,0.0,0.0,0.0,204.0,...,786.0,12.0,68.0,2377.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-12-05,0.0,94.0,0.0,0.0,3.0,0.0,0.0,14.0,184.0,...,755.0,10.0,35.0,2297.0,0.0,0.0,0.0,9.0,6.0,12.0


# 4. 데이터 결합

In [341]:
df = pd.merge(pivoted_kag, pivoted_roam, on='Date', suffixes=('_conf', '_roam'))

In [342]:
df.set_index('Date',inplace=True)

In [343]:
# 로밍데이터에는 있지만 글로벌 데이터에는 없거나, 반대인 경우는 삭제
del_cols = []
cols = list(df.columns)
for col in cols:
    if (col.find('_conf')  == -1 and col.find('_roam') == -1):
        del_cols.append(col)

In [344]:
del_cols[:5]

['BS', 'KM', 'CG', 'VA', 'XK']

In [345]:
df.drop(del_cols, axis=1, inplace=True)

In [360]:
df = df.astype(int)

In [361]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98 entries, 0 to 97
Columns: 308 entries, Date to target
dtypes: datetime64[ns](1), int32(307)
memory usage: 119.1 KB


# 국내 해외 유입 일별 확진자 데이터 병합

In [348]:
ov = pd.read_csv('./data/oversea_inflow.csv')

In [349]:
ov = ov[1:-1]

In [350]:
ov = ov.iloc[:,0:2]

In [351]:
ov.rename(columns={'date':'Date', 'oversea_inflow':'target'}, inplace=True)
ov['Date'] = pd.to_datetime(ov['Date'])

In [352]:
df = pd.merge(df, ov, on='Date')

In [362]:
df.head()

,Date,BO_conf,BN_conf,MM_conf,CF_conf,CD_conf,CI_conf,DO_conf,GM_conf,IR_conf,...,UG_roam,US_roam,UY_roam,UZ_roam,VN_roam,YE_roam,ZA_roam,ZM_roam,ZW_roam,target
0,2020-01-22,0,0,0,0,0,0,0,0,0,...,0,701,5,127,2592,0,12,0,0,0
1,2020-01-23,0,0,0,0,0,0,0,0,0,...,12,943,22,63,2338,0,21,9,3,0
2,2020-01-24,0,0,0,0,0,0,0,0,0,...,15,864,14,42,2251,0,0,0,0,1
3,2020-01-25,0,0,0,0,0,0,0,0,0,...,6,828,22,82,2336,0,12,0,0,0
4,2020-01-26,0,0,0,0,0,0,0,0,0,...,0,900,7,45,2695,0,6,0,0,1


In [355]:
df['target'] = df['target'].diff()

In [358]:
df['target'].iloc[0] = 0

In [360]:
df['target'] = df['target'].astype(int)

In [363]:
df.to_csv('final_1.csv')